<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Capstone проект №1. Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Catch Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

In [2]:
from __future__ import division, print_function
import datetime as dt
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score, roc_curve

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [3]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = ('/home/truename/Documents/capstone_user_identification')

In [4]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, '6c_w5_train_sessions.csv'), index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, '6c_w5_test_sessions.csv'), index_col='session_id')

In [5]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [6]:
train_test_df = pd.concat([train_df, test_df])

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
site1     253561 non-null int64
time1     253561 non-null object
site2     250098 non-null float64
time2     250098 non-null object
site3     246919 non-null float64
time3     246919 non-null object
site4     244321 non-null float64
time4     244321 non-null object
site5     241829 non-null float64
time5     241829 non-null object
site6     239495 non-null float64
time6     239495 non-null object
site7     237297 non-null float64
time7     237297 non-null object
site8     235224 non-null float64
time8     235224 non-null object
site9     233084 non-null float64
time9     233084 non-null object
site10    231052 non-null float64
time10    231052 non-null object
target    253561 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 42.6+ MB


In [8]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
site1     82797 non-null int64
time1     82797 non-null object
site2     81308 non-null float64
time2     81308 non-null object
site3     80075 non-null float64
time3     80075 non-null object
site4     79182 non-null float64
time4     79182 non-null object
site5     78341 non-null float64
time5     78341 non-null object
site6     77566 non-null float64
time6     77566 non-null object
site7     76840 non-null float64
time7     76840 non-null object
site8     76151 non-null float64
time8     76151 non-null object
site9     75484 non-null float64
time9     75484 non-null object
site10    74806 non-null float64
time10    74806 non-null object
dtypes: float64(9), int64(1), object(10)
memory usage: 13.3+ MB


**В обучающей выборке – 2297 сессий одного пользователя (Alice) и 251264 сессий – других пользователей, не Элис. Дисбаланс классов очень сильный, и смотреть на долю верных ответов (accuracy) непоказательно.**

In [10]:
train_df['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

**Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [11]:
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [12]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178
6,570,21,570,21,21,0,0,0,0,0
7,803,23,5956,17513,37,21,803,17514,17514,17514
8,22,21,29,5041,14422,23,21,5041,14421,14421
9,668,940,942,941,941,942,940,23,21,22


In [13]:
train_test_df_sites = (train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0)).astype('int')
top_sites = train_test_df_sites.stack().value_counts().index.tolist()[1:6]
print(top_sites)

[21, 23, 782, 22, 29]


In [14]:
train_test_df2 = train_test_df.fillna(0)
print(train_test_df2.shape)
train_test_df2.head(3)

(336358, 21)


,site1,site10,site2,site3,site4,site5,site6,site7,site8,site9,...,time1,time10,time2,time3,time4,time5,time6,time7,time8,time9
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2014-02-20 10:02:45,0,0,0,0,0,0,0,0,0
2,890,1518.0,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,...,2014-02-22 11:19:50,2014-02-22 11:20:16,2014-02-22 11:19:50,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:52,2014-02-22 11:19:52,2014-02-22 11:20:15
3,14769,14768.0,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,...,2013-12-16 16:40:17,2013-12-16 16:40:24,2013-12-16 16:40:18,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:20,2013-12-16 16:40:21,2013-12-16 16:40:22


In [16]:
train_test_df2['min_time'] = [min([dt.datetime.strptime(train_test_df2.iloc[row,t], '%Y-%m-%d %H:%M:%S') if train_test_df2.iloc[row,t]!= 0 else pd.NaT for t in [train_test_df2.columns.get_loc('time%d' % i) for i in range(1, 11)]]) for row in range(train_test_df2.shape[0])]
train_test_df2['max_time'] = [max([dt.datetime.strptime(train_test_df2.iloc[row,t], '%Y-%m-%d %H:%M:%S') if train_test_df2.iloc[row,t]!= 0 else pd.NaT for t in [train_test_df2.columns.get_loc('time%d' % i) for i in range(1, 11)]]) for row in range(train_test_df2.shape[0])]
train_test_df2.head(3)

,site1,site10,site2,site3,site4,site5,site6,site7,site8,site9,...,time2,time3,time4,time5,time6,time7,time8,time9,min_time,max_time
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,2014-02-20 10:02:45,2014-02-20 10:02:45
2,890,1518.0,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,...,2014-02-22 11:19:50,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:52,2014-02-22 11:19:52,2014-02-22 11:20:15,2014-02-22 11:19:50,2014-02-22 11:20:16
3,14769,14768.0,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,...,2013-12-16 16:40:18,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:20,2013-12-16 16:40:21,2013-12-16 16:40:22,2013-12-16 16:40:17,2013-12-16 16:40:24


In [17]:
train_test_df2['session_period'] = [(train_test_df2.iloc[row,train_test_df2.columns.get_loc('max_time')]-train_test_df2.iloc[row,train_test_df2.columns.get_loc('min_time')]).total_seconds() for row in range(train_test_df2.shape[0])]

In [18]:
train_test_df2['min_dt'] = [dt.datetime.strptime(t, '%Y-%m-%d %H:%M:%S') if t != 0 else pd.NaT for t in train_test_df2['time1']]
train_test_df2['min_dt_hour'] = [dt.datetime.strptime(t, '%Y-%m-%d %H:%M:%S').hour if t != 0 else 0 for t in train_test_df2['time1']]
train_test_df2['min_dt_weekday'] = [dt.datetime.strptime(t, '%Y-%m-%d %H:%M:%S').weekday() if t != 0 else 0 for t in train_test_df2['time1']]
train_test_df2.head(5)

,site1,site10,site2,site3,site4,site5,site6,site7,site8,site9,...,time6,time7,time8,time9,min_time,max_time,session_period,min_dt,min_dt_hour,min_dt_weekday
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,2014-02-20 10:02:45,2014-02-20 10:02:45,0.0,2014-02-20 10:02:45,10,3
2,890,1518.0,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,...,2014-02-22 11:19:51,2014-02-22 11:19:52,2014-02-22 11:19:52,2014-02-22 11:20:15,2014-02-22 11:19:50,2014-02-22 11:20:16,26.0,2014-02-22 11:19:50,11,5
3,14769,14768.0,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,...,2013-12-16 16:40:19,2013-12-16 16:40:20,2013-12-16 16:40:21,2013-12-16 16:40:22,2013-12-16 16:40:17,2013-12-16 16:40:24,7.0,2013-12-16 16:40:17,16,0
4,782,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,...,2014-03-28 10:54:42,2014-03-28 10:55:12,2014-03-28 10:55:42,2014-03-28 10:56:12,2014-03-28 10:52:12,2014-03-28 10:56:42,270.0,2014-03-28 10:52:12,10,4
5,22,178.0,177.0,175.0,178.0,177.0,178.0,175.0,177.0,177.0,...,2014-02-28 10:55:59,2014-02-28 10:55:59,2014-02-28 10:55:59,2014-02-28 10:57:06,2014-02-28 10:53:05,2014-02-28 10:57:11,246.0,2014-02-28 10:53:05,10,4


In [19]:
for j in range(len(top_sites)):
    train_test_df2['top_sites%d' % j] = train_test_df2[['site%d'%x for x in range(1,11)]].isin([top_sites[j]]).sum(axis=1)
train_test_df2['top_sites'] = train_test_df2[['site%d'%x for x in range(1,11)]].isin(top_sites).sum(axis=1)
train_test_df2.head(5)

,site1,site10,site2,site3,site4,site5,site6,site7,site8,site9,...,session_period,min_dt,min_dt_hour,min_dt_weekday,top_sites0,top_sites1,top_sites2,top_sites3,top_sites4,top_sites
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2014-02-20 10:02:45,10,3,0,0,0,0,0,0
2,890,1518.0,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,...,26.0,2014-02-22 11:19:50,11,5,0,0,0,0,0,0
3,14769,14768.0,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,...,7.0,2013-12-16 16:40:17,16,0,0,0,0,0,0,0
4,782,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,...,270.0,2014-03-28 10:52:12,10,4,0,0,10,0,0,10
5,22,178.0,177.0,175.0,178.0,177.0,178.0,175.0,177.0,177.0,...,246.0,2014-02-28 10:53:05,10,4,0,0,0,1,0,1


**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу *train_test_df_sites*, потом разделите обратно на обучающую и тестовую части.**

Обратите внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [20]:
def sparse_df(df):
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    X = df.values
    for d in X:
        for term in d:
            index = vocabulary.setdefault(term, len(vocabulary))
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))
    return csr_matrix((data, indices, indptr), dtype=int)[:,1:]

In [21]:
#train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')
train_test_df_sites = train_test_df2[['site%d' % i for i in range(1, 11)]+\
                                     ['min_dt_hour','min_dt_weekday','session_period']+ \
                                     #['top_sites']+\
                                     ['top_sites%d' % i for i in range(5)]\
                                    ].fillna(0).astype('int')

train_test_sparse = sparse_df(train_test_df_sites)
train_shares = train_df.shape[0]
X_train_sparse = train_test_sparse[:train_shares, :]
X_test_sparse = train_test_sparse[train_shares:, :]
y = train_df['target']

In [22]:
train_test_sparse.shape

(336358, 48371)

In [23]:
train_test_df_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,min_dt_hour,min_dt_weekday,session_period,top_sites0,top_sites1,top_sites2,top_sites3,top_sites4
session_id,,,,,,,,,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0,10,3,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518,11,5,26,0,0,0,0,0
3,14769,39,14768,14769,37,39,14768,14768,14768,14768,16,0,7,0,0,0,0,0
4,782,782,782,782,782,782,782,782,782,782,10,4,270,0,0,10,0,0
5,22,177,175,178,177,178,175,177,177,178,10,4,246,0,0,0,1,0


**1. Выведите размерности матриц *X_train_sparse* и *X_test_sparse* – 4 числа на одной строке через пробел: число строк и столбцов матрицы *X_train_sparse*, затем число строк и столбцов матрицы *X_test_sparse*.**

**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [24]:
with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'train_target.pkl'), 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl, protocol=2)

**Разобьем обучающую выборку на 2 части в пропорции 7/3, причем не перемешивая. Исходные данные упорядочены по времени, тестовая выборка по времени четко отделена от обучающей, это же соблюдем и здесь.**

In [25]:
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

**Создайте объект `sklearn.linear_model.SGDClassifier` с логистической функцией потерь и параметром *random_state*=17. Остальные параметры оставьте по умолчанию, разве что *n_jobs*=-1 никогда не помешает. Обучите  модель на выборке `(X_train, y_train)`.**

In [ ]:
#sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
#sgd_logit.fit(X_train,y_train)
from sklearn.grid_search import GridSearchCV
sgd = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
param_grid = {
    'n_iter': [1, 10, 20, 30, 50],
    'alpha': [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001],
    'tol': [0.01, 0.001, 0.0005, 0.0001],
    'shuffle': [True, False]
}

CV_rfc = GridSearchCV(estimator=sgd, param_grid=param_grid, cv= 10)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_)

In [ ]:
#{'alpha': 1e-05, 'n_iter': 10}
#sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17, n_iter=10)
a = 1.99526231497e-05
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17, n_iter=10, penalty = 'l2', alpha = a)
#sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17, n_iter=10, alpha=1e-05)
#sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17, n_iter=1, alpha=1e-06, shuffle=False, tol=0.01)
#{'alpha': 0.0001, 'n_iter': 10, 'shuffle': True, 'tol': 0.001}
#sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17, n_iter=10, alpha=1e-04, tol = 0.001, shuffle = True)
#{'alpha': 1e-05, 'n_iter': 10, 'shuffle': False, 'tol': 0.001}
#sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17, n_iter=1, alpha=1e-05, tol = 0.01, shuffle = False)
sgd_logit.fit(X_train,y_train)

**Сделайте прогноз в виде предсказанных вероятностей того, что это сессия Элис, на отложенной выборке *(X_valid, y_valid)*.**

In [ ]:
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)

**2. Напечатайте ROC AUC логистической регрессии, обученной с помощью стохастического градиентного спуска, на отложенной выборке. Округлите до 3 знаков после разделителя.**

In [ ]:
#print('shapes: ',logit_valid_pred_proba.shape,y_valid.fillna(0).astype('int').shape)
roc_auc_score(y_valid.fillna(0).astype('int'), logit_valid_pred_proba[:,1])

**Сделайте прогноз в виде предсказанных вероятностей отнесения к классу 1 для тестовой выборки с помощью той же *sgd_logit*, обученной уже на всей обучающей выборке (а не на 70%).**

In [ ]:
%%time
sgd_logit.fit(X_train_sparse,y)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse)

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
#from sklearn.metrics import accuracy_score
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
trees_grid = [5, 10, 15, 20, 30, 50, 75, 100]

for ntrees in trees_grid:
    rfc = RandomForestClassifier(n_estimators=ntrees, random_state=17, n_jobs=-1, oob_score=True)
    temp_train_acc = []
    temp_test_acc = []
    rfc.fit(X_train, y_train)
    temp_train_acc.append(rfc.score(X_train, y_train))
    temp_test_acc.append(rfc.score(X_valid, y_valid))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)

train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} trees".format(max(test_acc.mean(axis=1))*100, 
                                                        trees_grid[np.argmax(test_acc.mean(axis=1))]))

In [ ]:
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
max_depth_grid = [3, 5, 7, 9, 11, 13, 15, 17, 20, 22, 24]

# Обучаем на тренировочном датасете
for max_depth in max_depth_grid:
    rfc = RandomForestClassifier(n_estimators=100, random_state=17, n_jobs=-1, oob_score=True, max_depth=max_depth)
    temp_train_acc = []
    temp_test_acc = []
    rfc.fit(X_train, y_train)
    temp_train_acc.append(rfc.score(X_train, y_train))
    temp_test_acc.append(rfc.score(X_valid, y_valid))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)

train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} max_depth".format(max(test_acc.mean(axis=1))*100, 
                                                        max_depth_grid[np.argmax(test_acc.mean(axis=1))]))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, oob_score=True, n_jobs=-1, random_state=17, max_depth=3)
forest.fit(X_train,y_train)
y_predicted = forest.predict_proba(X_valid)
roc_auc_score(y_valid.fillna(0).astype('int'), y_predicted[:,1])

In [ ]:
parameters = {'max_features': [4, 7, 10, 13], 'min_samples_leaf': [1, 3, 5, 7], 'max_depth': [5,10,15,20]}
rfc = RandomForestClassifier(n_estimators=100, random_state=17, n_jobs=-1, oob_score=True)
gcv = GridSearchCV(rfc, parameters, n_jobs=-1, cv=skf, verbose=1)
gcv.fit(X_train, y_train)

In [ ]:
%%time
forest.fit(X_train_sparse,y)
forest_test_pred_proba = forest.predict_proba(X_test_sparse)

**Запишите ответы в файл и сделайте посылку на Kaggle. Дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera_Username", чтоб можно было легко идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard/public).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить.**

In [32]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
len(logit_test_pred_proba[:,1])

In [ ]:
write_to_submission_file(logit_test_pred_proba[:,1], '6c_week5_sgd_kaggle.csv')

In [ ]:
write_to_submission_file(forest_test_pred_proba[:,1], '6c_week5_sgd_kaggle.csv')

### Учетная запись на Kaggle - .developer.

In [27]:
from scipy.sparse import csr_matrix, hstack
#train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')
start_hour = pd.to_datetime(train_test_df2['time1']).apply(lambda x : x.hour).values.reshape(-1,1)
weekday =  pd.to_datetime(train_test_df2['time1']).apply(lambda x : x.weekday()).values.reshape(-1,1)
is_weekend = pd.to_datetime(train_test_df2['time1']).apply(lambda x : x.weekday()>= 5).values.reshape(-1,1)
is_start_in_morning = (start_hour < 12).reshape(-1,1)

train_test_df_sites = train_test_df2[['site%d' % i for i in range(1, 11)]+\
                                     ['min_dt_hour','min_dt_weekday','session_period']+ \
                                     #['top_sites']+\
                                     ['top_sites%d' % i for i in range(5)]\
                                    ].fillna(0).astype('int')

train_test_sparse = sparse_df(train_test_df_sites)
train_test_sparse_features = csr_matrix(hstack([train_test_sparse, start_hour, weekday, is_weekend, is_start_in_morning]))


train_shares = train_df.shape[0]
X_train_sparse = train_test_sparse_features[:train_shares, :]
X_test_sparse = train_test_sparse_features[train_shares:, :]
y = train_df['target']

In [30]:
#train_test_sparse_features = csr_matrix(hstack([train_test_sparse, start_hour, weekday, is_weekend, is_start_in_morning]))
X_train_sparse_features = train_test_sparse_features[:train_df.shape[0],:]
X_test_sparse_features = train_test_sparse_features[train_df.shape[0]:,:]
y = train_df.target.values
X_train, y_train = X_train_sparse_features[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse_features[train_share:, :], y[train_share:]

sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
sgd_logit.fit(X_train, y_train)

logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)
ra_score = roc_auc_score(y_valid, logit_valid_pred_proba[:,1])
print('%1.3f' % ra_score)

0.963


In [34]:
#sgd_logit = SGDClassifier(loss='log', random_state=17, n_jobs=-1)
a = 1.99526231497e-05
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17, n_iter=10, penalty = 'l2', alpha = a)
sgd_logit.fit(X_train, y_train)

logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)
ra_score = roc_auc_score(y_valid, logit_valid_pred_proba[:,1])
print('%1.3f' % ra_score)

#%%time
sgd_logit.fit(X_train_sparse_features, y)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse_features)

/home/truename/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.970


In [35]:
write_to_submission_file(logit_test_pred_proba[:,1], '6c_week5_sgd_kaggle.csv')

In [36]:
#sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
#sgd_logit.fit(X_train,y_train)
from sklearn.grid_search import GridSearchCV
sgd = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
param_grid = {
    'n_iter': [1, 10, 20, 30, 50],
    'alpha': [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001],
    'tol': [0.01, 0.001, 0.0005, 0.0001],
    'shuffle': [True, False]
}

CV_rfc = GridSearchCV(estimator=sgd, param_grid=param_grid, cv= 10)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_)

/home/truename/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/truename/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


{'alpha': 0.0001, 'shuffle': True, 'tol': 0.001, 'n_iter': 1}


In [40]:
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17, shuffle=True, n_iter=20, tol=0.001, penalty = 'l2', alpha = 0.0001)
sgd_logit.fit(X_train, y_train)

logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)
ra_score = roc_auc_score(y_valid, logit_valid_pred_proba[:,1])
print('%1.3f' % ra_score)

#%%time
sgd_logit.fit(X_train_sparse_features, y)
logit_test_pred_proba = sgd_logit.predict_proba(X_test_sparse_features)

/home/truename/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.961


## Критерии оценки работы:
- Правильные ли получились размерности матриц в п. 1? (max. 2 балла)
- Правильным ли получилось значения ROC AUC в п. 2? (max. 4 балла)
- Побит ли бенчмарк "sgd_logit_benchmark.csv" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)
- Побит ли бенчмарк "Logit +3 features" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в описание финального проекта (`html`, `pdf` или `ipynb`).
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям, отделив одного из пользователей от остальных – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов, также хороша [статья](https://alexanderdyakonov.wordpress.com/2017/03/10/cтекинг-stacking-и-блендинг-blending) Александра Дьяконова
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных веб-страницах Элис и остальными 1557 пользователями (*train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем большой тьюториал по Vowpal Wabbit и попробуем его в деле, на данных соревнования.